## BTC LOB Analysis - Creating and analysis dataframe (PostgreSQL)

In [ ]:
# Go to the terminal and create table
# psql -U mickael cryppro_v0

In [ ]:
import slumber
from datetime import datetime
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from decimal import Decimal as D
import logging
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import numpy as np
import time
import os
from sqlalchemy import create_engine
import psycopg2
import sys

class Exchange:
  def __init__(self, url, commissions):
    self.url = url
    self.commissions = commissions
    self.api = None
    self.session = None

  def get_requests_session(self):
    if self.session is not None:
      return self.session

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    self.session = s
    return s

  def get_slumber_api(self, append_slash=True):
    if self.api is not None:
      return self.api

    s = self.get_requests_session()
    self.api = slumber.API(self.url, session=s, append_slash=append_slash)
    return self.api

  def fetch(self, ticker):
    raise ValueError("Not implemented")

  def postprocess(self, ob1, ticker):
    raise ValueError("Not implemented")



class Kraken(Exchange):
  def __init__(self):
    super().__init__(
      url="https://api.kraken.com/0/public/",
      commissions = D('0.14')/100
    )

  def fetch(self, ticker):
    logging.debug('fetch kraken')

    api = self.get_slumber_api(append_slash=False)
    ob1 = api.Depth.get(pair=ticker, count=30)
    return ob1

  def postprocess(self, ob1, ticker):
    """
    ob1 - output of self.fetch(ticker)
    """
    logging.debug('postprocess kraken')
    
    if ob1['error']:
        raise ValueError("Kraken error: %s"%(ob1['error']))

    ob2 = copy.deepcopy(ob1)
    ob2 = list(ob2['result'].values())[0]
    for k1 in ['asks', 'bids']:
        ob2[k1] = pd.DataFrame(ob2[k1]).rename(columns={0:'price', 1:'amount', 2:'timestamp'})
        # convert to Decimal
        for k2 in ['price', 'amount']:
            ob2[k1][k2] = [D(x) for x in ob2[k1][k2].values]

    # commissions
    ob2['asks']['price pre-commission'] =  ob2['asks']['price']
    ob2['bids']['price pre-commission'] =  ob2['bids']['price']
    ob2['asks']['price'] = ob2['asks']['price'] * (1+self.commissions)
    ob2['bids']['price'] = ob2['bids']['price'] * (1-self.commissions)

    # set side
    ob2['asks']['side'] = 'ask'
    ob2['bids']['side'] = 'bid'

    ob3 = pd.concat([ob2['asks'], ob2['bids']], axis=0).reset_index(drop=True)
    ob3['source'] = 'kraken.com'
    ob3['ticker'] = ticker

    return ob3


kraken = Kraken()
ticker = 'XBTUSD'
order_book = kraken.fetch(ticker)
order_book = kraken.postprocess(order_book, ticker)
        
#Obtaining min_ask and max_bid for current timestamp
min_ask = order_book[order_book['side']=='ask']['price'].min()
max_bid = order_book[order_book['side']=='bid']['price'].max()

fn = '/home/mickael/db_pwd.secret'
pwd = open(fn, 'r').read().strip()
engine = create_engine("postgresql+psycopg2://mickael:%s@localhost/cryppro_v0"%pwd)
df2 = pd.read_sql_table('btcprice', con=engine)

data = pd.DataFrame({"minimum_ask": float(min_ask), "maximum_bid": float(max_bid), "timestamp": str(datetime.now()), "price": float((min_ask+max_bid)/2), "spread": float(min_ask-max_bid)}, index=['8'])

data.to_sql('btcprice', con=engine, if_exists='append')

In [ ]:
data

## Historical Analysis from database (table btcprice)

In [ ]:
fn = '/home/mickael/db_pwd.secret'
pwd = open(fn, 'r').read().strip()
engine = create_engine("postgresql+psycopg2://mickael:%s@localhost/cryppro_v0"%pwd)

df2 = pd.read_sql_table('btcprice', con=engine)

#Calculating Volatility
volatility_std = np.std(df2['spread'])
#pd.Series([float(x) for x in spread_array]).std()
volatility_var = np.var(df2['spread'])
print('Volatility with standard deviation: %8f'%volatility_std)    
print('and variance: %8f'%volatility_var)
    
#Plotting spread vs. volatility
fig1 = plt.figure(figsize=(10,10))
ax11 = fig1.add_subplot(111)
ax11.plot(range(0,len(df2['spread'])), df2['spread'], linewidth=4.0, color='b', label='Spread')
ax11.plot(range(0,len(df2['spread'])), df2['spread'].rolling(window=5).std(), linewidth=4.0, color='r', label='Volatility')    
ax11.set_xlabel('Time', fontsize=25)
ax11.set_ylabel('Spread and Volatility', fontsize=25)
ax11.set_title('Time Series', fontsize=25)
ax11.legend(fontsize = 25)
for item in (ax11.get_xticklabels() + ax11.get_yticklabels()):
    item.set_fontsize(fontsize=25)
plt.show()

In [ ]:
df2

In [ ]:
len(df2)